In [ ]:
import pysolr
import pandas as pd

In [ ]:
# url = "http://localhost:8983/solr/ckan"
url = "http://b2find.eudat.eu/solr"

In [ ]:
facet = "extras_TempCoverage"

solr_params = {
    'echoParams': 'none',
    'rows': 0,
    'wt': 'json',
    'q': '*',
     #'fq': "extras_TemporalCoverage:Begin",
    'fl': "extras_TemporalCoverageBeginDate",
    'indent': 'false',
    'facet': 'true',
    'facet.sort': 'index',
    'facet.limit': -1,
    'facet.field': ["extras_TemporalCoverageBeginDate", "extras_TemporalCoverageEndDate"]
}

In [ ]:
solr = pysolr.Solr(url)

In [ ]:
results = solr.search(**solr_params)

In [ ]:
# results.facets["facet_fields"]["extras_TemporalCoverageBeginDate"]

In [ ]:
result_begin_counts = results.facets["facet_fields"]["extras_TemporalCoverageBeginDate"]
x_begin = result_begin_counts[0::2]
y_end = result_begin_counts[1::2]


In [ ]:
df_begin = pd.DataFrame(list(zip(x_begin, y_end)), columns =['date', 'counts'])
df_begin

In [ ]:
df_begin["date"]=df_begin["date"].str[:-16]
df_begin["date"]=pd.to_numeric(df_begin["date"], downcast="signed")
df_begin

In [ ]:
df_begin = df_begin.groupby(pd.Grouper(key="date")).sum()
df_begin

In [ ]:
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool
from bokeh.plotting import figure

source = ColumnDataSource(df_begin[250:300])

p = figure(plot_height=300, plot_width=800, toolbar_location=None,
           # x_axis_type="datetime", 
           #x_axis_location="above",
           #y_axis_type="log",
           #y_range=(1.0, 10**5),
           background_fill_color="#efefef")

p.line('date', 'counts', source=source)
p.yaxis.axis_label = 'Number of Datasets per Year'

show(p)